In [1]:
#Import relevant modules.
import pygsti
import numpy as _np

import matplotlib.pyplot as plt
%matplotlib inline

import time
import os

basedir = '/Users/phumphreys/Repositories/analysis/notebooks/Quantum Networks/Simulations/GST'

In [2]:
#Create maximum lengths list
maxLengths = [0,1,2,4,8,16,32]
import pickle
pickle.dump( maxLengths, open(os.path.join(basedir,"MyMaxLengths.pkl"), "wb"))

In [6]:
#Construct a target gateset
gs_target = pygsti.construction.build_gateset([2],[('Q0',)], [u'Gx',u'Gy',u'Gu',u'Gv'], 
                                             [ "X(pi/2,Q0)", "Y(pi/2,Q0)","X(pi,Q0)","Y(pi,Q0)"],
                                             prepLabels=['rho0'], prepExpressions=["0"],
                                             effectLabels=['E0'], effectExpressions=["1"], 
                                             spamdefs={'plus': ('rho0','E0'), 'minus': ('rho0','remainder') } )


In [16]:
# Load in fiducials, germs, maxLengths:
measFidList = pygsti.io.load_gatestring_list(os.path.join(basedir,"MeasFiducials.txt"))
prepFidList = pygsti.io.load_gatestring_list(os.path.join(basedir,"PrepFiducials.txt"))
germList = pygsti.io.load_gatestring_list(os.path.join(basedir,"MyGerms.txt"))
maxLengths = pickle.load( open(os.path.join(basedir,"MyMaxLengths.pkl")))

#Create a list of GST experiments for this gateset, with
#the specified fiducials, germs, and maximum lengths
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target.gates.keys(), prepFidList, measFidList, germList, maxLengths)

#Create an empty dataset file, which stores the list of experiments
#plus extra columns where data can be inserted
pygsti.io.write_empty_dataset(os.path.join(basedir,"MyDataTemplate.txt"), listOfExperiments,
                              "## Columns = plus count, count total")

In [17]:
#Create a gateset of depolarized gates and SPAM relative to target, and generate fake data using this gateset.
gs_datagen = gs_target.depolarize(gate_noise=0.05, spam_noise=0.001)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=10000,
                                            sampleError="binomial", seed=2015)

#Save our dataset
pygsti.io.write_dataset(os.path.join(basedir,"MyDataset.txt"), ds)

In [18]:
#Note; to load the dataset back again, do:
#ds = pygsti.io.load_dataset("tutorial_files/MyDataset.txt")

gs_target.set_all_parameterizations("TP")
results = pygsti.do_long_sequence_gst(os.path.join(basedir,"MyDataset.txt"), gs_target, 
                                        prepFidList, measFidList, germList, maxLengths,
                                        gaugeOptParams={'itemWeights': {'spam': 1e-3, 'gates': 1.0}})

s = pickle.dumps(results)
r2 = pickle.loads(s)
print(r2.gatesets['final estimate'])

Loading /Users/phumphreys/Repositories/analysis/notebooks/Quantum Networks/Simulations/GST/MyDataset.txt: 100%
Writing cache file (to speed future loads): /Users/phumphreys/Repositories/analysis/notebooks/Quantum Networks/Simulations/GST/MyDataset.txt.cache
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.24274776764
  1.34207600445
  1.18208537542
  1.17564835891
  0.0109927204348
  0.00977036148705
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.24264068712
  1.41421356237
  1.41421356237
  1.41421356237
  1.73856273296e-16
  3.24754494531e-17
  
--- Iterative MLGST: Iter 1 of 7  142 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
   groups of ~1 procs each, to distribute over 55 params (taken as 1 param groups of ~55 params).
  Sum of Chi^2 = 105.441 (142 data params - 43 model params = expected mean of 99; p-value = 0.310191)
  Completed

In [19]:
results.create_brief_report_pdf(confidenceLevel=95, filename=os.path.join(basedir,"easy_brief.pdf"), verbosity=2)


    
--- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
   22s           0.0083130551
   30s           0.0083058707
   38s           0.0082997856
   45s           0.0082992779
  The resulting min sqrt(sum(gateCIs**2)): 0.00829928
*** Generating tables ***
  Generating table: bestGatesetSpamTable (w/95% CIs)  [0.0s]
  Generating table: bestGatesetSpamParametersTable (w/95% CIs)  [0.0s]
  Generating table: bestGatesetGatesTable (w/95% CIs)  [0.0s]
  Generating table: bestGatesetDecompTable (w/95% CIs)  [0.3s]
  Generating table: bestGatesetRotnAxisTable (w/95% CIs)  [0.4s]
  Generating table: bestGatesetVsTargetTable (w/95% CIs)  [0.2s]
  Generating table: bestGatesetErrorGenTable (w/95% CIs)  [0.0s]
  Generating table: progressTable (w/95% CIs)  [0.3s]
*** Generating plots ***
*** Merging into template file ***
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial output PDF /Users/phumphreys/Repositories/analysis/notebooks/Quantum Networks/Si

In [ ]:
#create a full GST report (most detailed and pedagogical; best for those getting familiar with GST)
results.create_full_report_pdf(confidenceLevel=95, filename=os.path.join(basedir,"easy_full.pdf"), verbosity=2)